<b><img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250, style="padding: 10px"> 
<p><p><p><p><p><p>
<b>Interactive Catalog Visualization</b> <br>
Last verified to run on 2022-05-31 with LSST Science Pipelines release w_2022_22<br>
Contact authors: Leanne Guy <br>
Target audience: All DP0 delegates. <br>
Minimum Container Size: medium <br>
Questions welcome at <a href="https://community.lsst.org/c/support/dp0">community.lsst.org/c/support/dp0</a> <br>
Find DP0 documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a> <br>

**Credit:** This tutorial was inspired by a notebook originally developed by Keith Bechtol in the context of the LSST Stack Club. It has been updated and extended for DP0.1 by Leanne Guy. Please consider acknowledging Leanne Guy and Keith Bechtol in any publications or software releases that make use of this notebook's contents.

### Learning Objectives

This tutorial, together with tutorial `08a_Interactive_Image_Visualization`, introduces three open-source Python libraries that enable powerful interactive visualization of catalogs.
 1. [**HoloViews**](http://holoviews.org): Produce high-quality interactive visualizations easily by annotating plots and images rather than using direct calls to a plotting library
 2. [**Bokeh**](https://bokeh.org): A powerful data visualization library that provides interactive tools including brushing and linking between multiple plots. `Holoviews` + `Bokeh`
 3. [**Datashader**](https://datashader.org): Accurately render very large datasets quickly and flexibly.
 
These packages are part of the [Holoviz](http://holoviz.org/) ecosystem of tools intended for visualization in a web browser and can be used to create quite sophisticated dashboard-like interactive displays and widgets. The goal of this tutorial is to provide an introduction and starting point from which to create more advanced, custom interactive visualizations. Holoviz has a [vibrant and active community](https://discourse.holoviz.org/) where you can ask questions and discuss vizualizations with a global community. 

### Logistics
This notebook is intended to be runnable on `data.lsst.cloud`. Note that occasionally the notebook may seem to stall, or the interactive features may seem disabled. If this happens, usually a restart of the kernel fixes the issue. You might also need to log out of the RSP and start a "large" instance of the JupyterLab environment. In some examples shown in this notebook, the order in which the cells are run is important for understanding the interactive features, so you may want to re-run the set of cells in a given section if you encounter unexpected behavior. Note that some of the examples require manual selection of points on a graph to run correctly.

### Setup

In [ ]:
# General python imports
import numpy as np
import pandas as pd
import warnings

# Update this option setting as you prefer
pd.set_option('display.max_rows', 5)

# Astropy
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.units.quantity import Quantity

# LSST imports
from lsst.rsp import get_tap_service

# Bokeh and Holoviews for visualization
import bokeh
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, Range1d, HoverTool
from bokeh.models import CDSView, GroupFilter
from bokeh.plotting import figure, gridplot
from bokeh.transform import factor_cmap

import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import datashade, dynspread
from holoviews.plotting.util import process_cmap

import datashader as dsh

# Set the holoviews plotting library to be bokeh
# You will see the holoviews + bokeh icons displayed when the library is loaded successfully
hv.extension('bokeh')

# Display bokeh plots inline in the notebook
output_notebook()

If a message about \"Patching auth into notebook.base.handlers ...\" appeared above, it is ok to ignore, as are messages about the number of threads.

In [ ]:
# What versions of bokeh and holoviews nd datashader are we working with?
# This is important when referring to online documentation as
# APIs can change between versions.
print("Bokeh version: " + bokeh.__version__)
print("Holoviews version: " + hv.__version__)
print("Datashader version: " + dsh.__version__)

In [ ]:
# Ignore warnings
import warnings
from astropy.units import UnitsWarning
warnings.simplefilter("ignore", category=UnitsWarning)

In [ ]:
# What version of the LSST Science Pipelnes are we using?
! echo $IMAGE_DESCRIPTION
! eups list -s | grep lsst_distrib

### 1. Data preparation

The basis for any data visualization is the underlying data. In this tutorial we will work with tabular data.

#### 1.1 DP0.1 tabular dataset
We will execute a cone search about a defined coordinate with a specified radius using the Rubin TAP service. For more details about using the TAP service and ADQL queries, please refer to tutorial `02_Intermediate_TAP_Query`.

In [ ]:
# Get a Rubin TAP service instance. 
service = get_tap_service()
assert service is not None

In [ ]:
# Define a function to build a query passing a coordinate and a search radius
def getQuery(c: SkyCoord, r: Quantity) -> str:
    query = "SELECT obj.ra, obj.dec, obj.objectId, obj.extendedness, "\
            "obj.mag_g_cModel, obj.mag_r_cModel, obj.mag_i_cModel, "\
            "truth_type, truth.match_objectId " \
            "FROM dp01_dc2_catalogs.object as obj " \
            "JOIN dp01_dc2_catalogs.truth_match as truth " \
            "ON truth.match_objectId = obj.objectId " \
            "WHERE CONTAINS(POINT('ICRS', obj.ra, obj.dec),"\
            "CIRCLE('ICRS', " + str(c.ra.value) + ", " + str(c.dec.value) + ", " \
            + str(r.to(u.deg).value) + " )) = 1 " \
            "AND obj.good = 1 "  \
            "AND truth.match_objectid >= 0 " \
            "AND truth.is_good_match = 1"
    return query

In [ ]:
# Define a reference position on the sky and a radius in degrees for a cone search
c1 = SkyCoord(ra=55.65*u.degree, dec=-40.*u.degree, frame='icrs')
radius = 4 * u.deg

In [ ]:
query = getQuery(c1, radius)

Warnings about units that might appear above are ok to ignore.

This query will return a very large dataset, so we will use an asynchronous query. This could take a few minutes.

In [ ]:
job = service.submit_job(query)

In [ ]:
job.run()

In [ ]:
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
%%time
data = job.fetch_result().to_table().to_pandas()

In [ ]:
assert len(data) == 23155236

In [ ]:
# Compute some colours 
data['gmi'] = data['mag_g_cModel'] - data['mag_i_cModel']
data['rmi'] = data['mag_r_cModel'] - data['mag_i_cModel']
data['gmr'] = data['mag_g_cModel'] - data['mag_r_cModel']

In [ ]:
# Map the truth type to a descriptive string
# Catalog schema for the truth table can be found at:
#  https://dp0-1.lsst.io/data-products-dp0-1/index.html#catalogs
data['truth_type'] = data['truth_type'].map(
    {1: 'galaxy', 2: 'star', 3: 'SNe'})

In [ ]:
assert data[data["truth_type"] == "star"].shape[0] == 504174
assert data[data["truth_type"] == "galaxy"].shape[0] ==  22651062
assert data[data["truth_type"] == "SNe"].shape[0] == 0

We will create a small subsample of this dataset of about 20000 points to demenstrate some of the basic functionality of the `holoviz` tools. In section 6, we will look at how to effectively visualize the full dataset of 200 million points. 

In [ ]:
frac = 0.001
data20K = data.sample(frac=frac, axis='index')
assert len(data20K) == round(frac * len(data))

### 2. Holoviews

[Holoviews](https://holoviews.org) supports easy analysis and visualization by annotating data rather than utilizing direct calls to plotting packages. For this tutorial, we will use [Bokeh](hrrps://bokeh.org) as the plotting library backend for Holoviews. This is defined in the `Setup` section above with the `hv.extension('bokeh')` call.  Holoviews supports several plotting libraries and there is an exercise to the user at the end of this section to explore using Holoviews with other plotting packages. 

#### 2.1 Visualizing tabular data with Holoviews

The basic core primitives of Holoviews are [Elements](http://holoviews.org/Reference_Manual/holoviews.element.html); hv.Element. Elements are simple wrappers around your data that provide a semantically meaningful visual representation. An Element may be a set of Points, an Image, a Curve, a Histogram, etc. See the Holoviews [Reference Gallery](http://holoviews.org/reference/index.html) for all the various types of Elements that can be created with Holoviews. 

In this first example we will use the Holoviews [Scatter Element](http://holoviews.org/reference/elements/bokeh/Scatter.html) to quickly visualize the catalog data retrieved in section 1 as a scatter plot. HoloViews maintains a strict separation between content and presentation. This separation is achieved by maintaining sets of keyword values as `options` that specify how `Elements` are to appear.  In this first example we will apply the default options and remove the toolbar. 

In [ ]:
# Make a simple scatter plot of the data using the Scatter element. 
hv.Scatter(data20K).options(toolbar=None)

The `data20K` set contains many columns. If no columns are specified, explicitly the first 2 columns are taken for x and y respectively by the [Scatter Element](https://holoviews.org/reference/elements/bokeh/Scatter.html).

Now let's bin the data in RA using the robust [Freedman Diaconis Estimator](https://numpy.org/doc/stable/reference/generated/numpy.histogram_bin_edges.html#numpy.histogram_bin_edges) and plot
the resulting distribution using the Holoviews [Histogram Element](http://holoviews.org/reference/elements/bokeh/Histogram.html). 
We will also add in some basic plot options. Read more about about [customizing plots](https://holoviews.org/user_guide/Customizing_Plots.html) via `options`. Note that `options` can be shortened to `opts`.

In [ ]:
(ra_bin, count) = np.histogram(data20K['ra'], bins='fd')
ra_distribution = hv.Histogram((ra_bin, count)).opts(
    title="RA distribution",color='darkmagenta', 
    xlabel='RA', fontscale=1.2,
    height=400, width=400)

In [ ]:
ra_distribution

Next, let's create a layout of several plots. A `Layout` is a type of `Container` that can contain any HoloViews object. Other types of Containers that exist include `Overlay`, `Gridspace`, `Dynamicmap`, etc. See the Holoviews [Reference Gallery](http://holoviews.org/reference/index.html) for the full list of `Layouts` that can be created with Holoviews. See [Building Composite Objects](http://holoviews.org/user_guide/Building_Composite_Objects.html) for the full details about the ways Containers can be composed.

The `+` operator is used to create a Layout. 

In [ ]:
# Slice the data and set some moore options
skyplot = hv.Scatter(data20K[["ra", "dec"]]).opts(
    title="Skyplot", toolbar='above', tools = ['hover'], 
    height=350, width=350, alpha=0.2, size=2) 

# Construct a layout using the `+` operator
skyplots = skyplot + \
           ra_distribution.options(height=350, width=350)

In [ ]:
skyplots

Note that these two plots are not linked, they are two independent plots layed out next to each other. Try zooming in on the skyplot, you will notice that the data are not rebinned in the RA Distribution plot.  We will see how to link plots in Section 3. The tools however do apply to both plots. Try modifying both plots and then use the `reset` tool. Notice that both plots are reset to their original states. 

Next, let's setup some default plot options to avoid duplicating long lists everytime we want to make a plot. As different plotting packages typically provide different customization capabilities, we will define one set of options for a Bokeh backend and one for a matplotlib backend.

In [ ]:
# Bokeh specific customizations as a python dictionary 
plot_style_bkh = dict(alpha=0.5, color='darkmagenta', 
                      marker='triangle', size=3,
                      xticks=5, yticks=5,
                      height=400, width=400, 
                      toolbar='above')

# Matplotlib specific customizations
plot_style_mpl = dict(alpha=0.2, color='c', marker='s', 
                      fig_size = 200, s=2, 
                      fontsize=14, xticks=8, yticks=8)

Instead of subsetting a dataset to choose which columns to plot, Holoviews allows us to specifiy the dimensionality directly. 
`kdims` are the key dimensions or the independent variable(s) and `vdims` are the value dimensions or the dependenent variable(s). 

In [ ]:
# Use the bokeh plot style
plot_style = plot_style_bkh

In [ ]:
hv.Scatter(data20K, 
           kdims=['gmi'], vdims=['mag_g_cModel']
          ).opts(invert_yaxis=True,
                 xlabel=" (g-i)", ylabel="g",
                 **plot_style)

The dimensions have be specified as strings above, but they are in fact rich objects. Dimension objects support a long descriptive label, which complements the short programmer-friendly name. Let's look at color-color diagram of the stars in the dataset.

In [ ]:
# Axes as rich objects
rmi = hv.Dimension('rmi', label='(r-i)', range = (-0.8,3.0))
gmr = hv.Dimension('gmr', label='(g-r)', range = (-0.8,3.0))

Let's make a colour-colour scatter plot of just the stars in the dataset and also display the distribution of samples along both value dimensions using the `hist()` method of the [Scatter Element](http://holoviews.org/reference/elements/bokeh/Scatter.html).

In [ ]:
stars = data20K[data20K["truth_type"] == 'star']
col_col = hv.Scatter(stars, kdims=gmr, 
                     vdims=rmi).opts(**plot_style)

#  Use the hist method to show the distribution of samples along both value dimensions.
col_col = col_col.hist(dimension = [gmr,rmi], 
                       num_bins=100, adjoin=True)

In [ ]:
col_col

Try zooming in on regions of the plot. The histograms are automatically recomputed.  

The techniques to apply customizations in the cells above use standard Python syntax and are the recommended way to customize your visualizations in HoloViews. HoloViews also supports IPython magic commands. Magics are a much older approach that is not standard Python and is specific to notebooks. [HoloViews notebook magic](https://holoviews.org/user_guide/Notebook_Magics.html) supports both line and cell magics. Here is an example of using magics to plot the same spatial distribution of `Objects` as above.

In [ ]:
%%opts Scatter [tools=['hover'], toolbar='above',height=400, width=400](color='grey')
hv.Scatter(data20K).opts(alpha=0.3, size=3)

Our result set above contained a lot of columns. Often we want to be selective about which information we show in the hover tool and customize the names. We do this by creating a custom hover tool. 

In [ ]:
raDecHover = HoverTool(
    tooltips=[
        ( 'ra,dec', '@ra / @dec'),
        ( 'rmag', '@mag_r_cModel'),
        ( 'type', '@truth_type'),
    ],
    formatters={
        'ra/dec' : 'printf',
        'rmag' : 'numeral',
        'type' : 'printf',
    },
    point_policy="follow_mouse"
)

In [ ]:
hv.Scatter(data20K).opts(tools=[raDecHover], 
                      **plot_style_bkh)

It might be necessary to zoom in until points are distinguishable in order to notice how the hover tool box contents list only three properties per point.

### 3.0 Linked plots and data brushing with Bokeh

A very useful feature of `Bokeh` is the ability to add connected interactivity between plots that show different attributes of the same data. This is called `linking`. With linked plots, we can carry out data `brushing`, whereby we select and maniplulate data synchronously across multiple linked plots. For example, if we link a skyplot with a colour-magnitude diagram of the same dataset, we can interactively explore the relationship between the positions of objects in each plot.  

We will now look at the Bokeh plotting library directly to demonstrate how to set up brushing and linking between two panels showing different repsentations of the same dataset. A selection applied to either panel will highlight the selected points in the other panel.

This section is based on [Bokeh linked brushing](http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/linking.html#linked-brushing).

#### 3.1 Data preparation
The basis for any data visualization is the underlying data. Getting the data preparation phase right is key to creating powerful visualizations. 
Bokeh works with a ColumnDataSource (CDS).  A CDS is essentially a collection of sequences of data that have their own unique column name. We will create a CDS from the data returned by the query above and pass it directly to bokeh. The CDS is the core of bokeh plots. Bokeh automatically creates a CDS from data passed as python lists or numpy arrays.  CDS are useful as they allow data to be shared between multiple plots and renderers, enabling brushing and linking. 

In [ ]:
# Create a column data source for the plots to share. 
col_data = dict(x0=data20K['ra'] - c1.ra.value,
                y0=data20K['dec'] - c1.dec.value,
                x1=data20K['gmr'],
                y1=data20K['mag_g_cModel'],
                ra=data20K['ra'], dec=data20K['dec'])
source = ColumnDataSource(data=col_data)

# Additional data can be added to the CDS after creation
source.data['objectId'] = data20K['objectId']
source.data['rmi'] = data20K['rmi']
source.data['gmr'] = data20K['gmr']
source.data['mag_r_cModel'] = data20K['mag_r_cModel']
source.data['truth_type'] = data20K['truth_type']
source.data['extendedness'] = data20K['extendedness']

# Create a view on truth_type stars
stars = CDSView(source=source,
                filters=[GroupFilter(column_name='truth_type', group="star")])

#### 3.2 Colour-Magnitude Diagram linked to a Skyplot

We will use bokeh to plot a color-magnitude (g vs. g-i) diagram making use of the cModel magnitudes and a skyplot and then link the two.

In [ ]:
# Create a custom hover tool for each panel. 
hover_left = HoverTool(tooltips=[("ObjectId", "@objectId"),
                                 ("(ra,dec)", "(@ra, @dec)"),
                                 ("type", "@truth_type")
                                 ])
hover_right = HoverTool(tooltips=[("ObjectId", "@objectId"),
                                  ("(g-r,g)", "(@x1, @y1)"),
                                  ("extendedness", "@extendedness")
                                  ])
tools = "box_zoom,box_select,lasso_select,reset,help"
tools_left = [hover_left, tools]
tools_right = [hover_right, tools]

In [ ]:
# Create a new plot and add a renderer. We will look at stars only
stars = CDSView(source=source,
                filters=[GroupFilter(column_name='truth_type', group="star")])

left = figure(tools=tools_left, 
              plot_width=400, plot_height=400,
              title='Spatial: Centered on (RA, Dec) = (%.2f, %.2f)' %
              (c1.ra.value, c1.dec.value))
left.circle('x0', 'y0', hover_color='firebrick', 
            size=3, alpha=0.7,
            source=source, 
            view=stars # select only stars
           )
left.x_range = Range1d(5.5, -5.5)
left.y_range = Range1d(-5.5, 5.5)
left.xaxis.axis_label = 'Delta ra'
left.yaxis.axis_label = 'Delta dec'

# create another new plot and add a renderer
right = figure(tools=tools_right, plot_width=400, plot_height=400, 
               title='CMD')
right.circle('x1', 'y1', hover_color='firebrick', 
             size=4, alpha=0.8,
             source=source, 
             view=stars  # Select only stars
            )
right.x_range = Range1d(-1.5, 2.8)
right.y_range = Range1d(32., 16.)
right.xaxis.axis_label = '(g-r)'
right.yaxis.axis_label = 'g'

In [ ]:
# Display the grid of plots
p = gridplot([[left, right]])
show(p)

Use the hover tool to see information about individual datapoints (e.g., the `ObjectId`). This information should appear automatically as you hover the mouse over the datapoints. Notice the data points highlighted in red on one panel with the hover tool are also highlighted on the other panel.

Next, click on the selection box icon (with a "+" sign) or the selection lasso icon found in the upper right corner of the figure. Use the selection box and selection lasso to make various selections in either panel by clicking and dragging on either panel. The selected data points will be displayed in the other panel.

### 4.0 Further analysis with Holoviews Linked Streams

If we want to do subsequent calculations with the set of selected points, we can use HoloViews linked streams for custom interactivity. The following visualization is a modification of this example. As for the example above, use the selection box and selection lasso to datapoints on the left panel. The selected points should appear in the right panel. Finally, notice that as you change the selection on the left panel, the mean x- and y-values for selected datapoints are shown in the title of right panel.

Based on [Holoviews Selection1D points](http://holoviews.org/reference/streams/bokeh/Selection1D_points.html)

In [ ]:
# Declare some points
points = hv.Points((data20K['ra'] - c1.ra.value, data20K['dec'] - c1.dec.value)
                  ).options(tools=['box_select', 'lasso_select'])

# Declare points as source of selection stream
selection = streams.Selection1D(source=points)

# Define a function that uses the selection indices to slice points and compute stats
def selected_info(index) -> str:
    selected = points.iloc[index]
    if index:
        label = 'Mean x, y: %.3f, %.3f' % tuple(selected.array().mean(axis=0))
    else:
        label = 'No selection'
    return selected.relabel(label).options(color='red')


# Combine points and DynamicMap
# Notice the syntax used here: the "+" sign makes side-by-side panels
points + hv.DynamicMap(selected_info, streams=[selection])

In the next cell, we access the indices of the selected datapoints. We could use these indices to select a subset of full sample for further examination.

In [ ]:
print(len(selection.index))

### 5.0  Visualizing Larger Datasets with Datashader

The interactive features of Bokeh work well with datasets up to a few tens of thousands of data points. To efficiently explore larger datasets, we'd like to use another visualization model that offers better scalability, namely Datashader.

In the examples below, notice that as one zooms in on the datashaded two-dimensional histograms, the bin sizes are dynamically adjusted to show finer or coarser granularity in the distribution. This allows one to interactively explore large datasets without having to manually adjust the bin sizes while panning and zooming. Zoom in all the way and you can see individual points (i.e., bins contain either zero or one count). If you zoom in far enough, the individual points are represented by extremely small pixels in datashader that are difficult to see. A solution is to dynspread instead of datashade, which will preserve a finite size of the plotted points.

The next cell also uses the concept of linked Streams in HoloViews for custom interactivity, in this case to create a selection box. We'll use that selection box tool in the following cell.

#### 5.1 Color-color plot 

Here we plot a color-colour diagram of the cModel magnitudes obtained from the query in 1. Data Preparation

In [ ]:
# Create color-color plot using bokeh
plot_options = {'plot_height': 400, 'plot_width': 800,
                'tools': ['pan','box_zoom','box_select', 'wheel_zoom','reset', 'help']}

p = figure(title="Colour-Colour Diagram (cModel magnitudes)",
           x_axis_label="(g-r)", y_axis_label="(r-i)",
           x_range=(-2.0, 3.0), y_range=(-2.0, 3.0),
           **plot_options)
p.circle(x='gmr', y='rmi', source=source,
         size=2, alpha=0.2,
         hover_color='firebrick',
         legend_field="truth_type",
         color=factor_cmap('truth_type', 'Category10_3',
                           ['galaxy', 'star', 'SNe']))

# Add a custiomized hover tool
hover = HoverTool(tooltips=[("objectId", "@objectId"),
                            ("(ra,dec)", "(@ra, @dec)"),
                            ("(g-r,r-i)", "(@gmr, @rmi)"),
                            ("type", "@truth_type")])
p.add_tools(hover)

In [ ]:
show(p)

We see that even with a medium sized dataset of ~20K points, this plot suffers from overplotting.  A classic strategy is to specify transparency of the glyphs so we can better see sparse and dense areas. In the plot above we have `alpha=0.2`. This helps,  but washes out the detail in the sparser regions. An additional problem is that we cannot add too many glyphs to any plot. 

Holoviews + Datashader allow us to plot millions to billions of points to produce much more informative plots. DataShader rasterizes or aggregates datasets into regular grids that can then be further analysed or viewed as plots or images. 

In [ ]:
# Create a holoviews object to hold and plot data
points = hv.Points((source.to_df()['gmr'], 
                    source.to_df()['rmi'])).opts(
    tools=['box_select', 'lasso_select'])


# Create the linked streams instance
boundsxy = (0, 0, 0, 0)
box = streams.BoundsXY(source=points, bounds=boundsxy)
bounds = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box])

# Apply the datashader
p = dynspread(datashade(points, cmap="Viridis"))
p = p.opts(width=800, height=300,
    padding=0.05, show_grid=True,
    xlim=(-8.0, 18.0), ylim=(-18.0, 8.0),
    xlabel="(g-r)", ylabel="(r-i)")

In [ ]:
# Render the datashaded plot
p * bounds

This `datashade` plot of the same color-color diagram as above does not require any magic-number parameters such as size and alpha and automatically ensures that there is no saturation or overplotting.  Select the `wheel zoom` and adjust the image as you interact with the plot. Note how the shades of color of the data points change according to the local density.

In fact, this dataset is really too small to use datashader on. Let's see how to visualize the full 200 million dataset returned by the query with datashader. 

In [ ]:
# Create a Points Element for the data
points = hv.Points((data['gmr'], 
                    data['rmi'])).opts(
    tools=['box_select', 'lasso_select'])

# Create the linked streams instance
boundsxy = (0, 0, 0, 0)
box = streams.BoundsXY(source=points, bounds=boundsxy)
bounds = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box])

# Apply the datashader
p = dynspread(datashade(points, cmap="Viridis"))
p = p.opts(width=800, height=300,
    padding=0.05, show_grid=True,
    xlim=(-8.0, 18.0), ylim=(-18.0, 8.0),
    xlabel="(g-r)", ylabel="(r-i)")

In [ ]:
%%time
# Render the datashaded plot
p * bounds

#### 5.2 Adding a callback function
Next we will add callback functionality to the colour-colour diagam above to retrieve the indices of selected points. We use the box selection tool to create a selection box for a two-dimensional histogram and then count the number of datapoints within the selection region.

> STOP - Select some data points from the plot above using the box select tool before proceeding

In [ ]:
selection = (points.data.x > box.bounds[0]) \
    & (points.data.y > box.bounds[1]) \
    & (points.data.x < box.bounds[2]) \
    & (points.data.y < box.bounds[3])
print('The selection box contains %i datapoints'%(np.sum(selection)))

#### 5.3 Linked plots with interactive selection

Now we will plot a spatial distribution on the sky of all the data (in the left-hand plot below), and link it to a two-dimensional histogram of the data (right-hand plot below). Use a box selection in the spatial distribution to change which data are included in the histogram.

In [ ]:
# First, create a holoviews dataset instance. 
# Here we label some of the columns.
kdims = [('ra', 'RA(deg)'), ('dec', 'dec(deg)')]
vdims = [('mag_r_cModel', 'r(mag)')]
ds = hv.Dataset(data, kdims, vdims)

In [ ]:
points = hv.Points(ds)
boundsxy = (np.min(ds.data['ra']), np.min(ds.data['dec']),
            np.max(ds.data['ra']), np.max(ds.data['dec']))
box = streams.BoundsXY(source=points, bounds=boundsxy)
box_plot = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box])

In [ ]:
# Custom callback functionality to update the linked histogram
def log_inf(x) -> float:
    return np.log(x) if x > 0 else 0


def update_histogram(bounds=bounds) -> hv.Histogram:
    selection = (ds.data['ra'] > bounds[0]) & \
                (ds.data['dec'] > bounds[1]) & \
                (ds.data['ra'] < bounds[2]) & \
                (ds.data['dec'] < bounds[3])

    selected_mag = ds.data.loc[selection]['mag_r_cModel']
    frequencies, edges = np.histogram(selected_mag)
    hist = hv.Histogram((list(map(log_inf, frequencies)), edges))
    return hist

In [ ]:
dmap = hv.DynamicMap(update_histogram, streams=[box]).options(
    height=400, width=400)
datashade(points,
          cmap=process_cmap("Viridis", provider="bokeh")) * \
box_plot.options(height=400, width=400) + \
dmap

Try changing the box selection and watch as the histogram is recomputed and displayed. 

In [ ]:
# Finally, delete the job
job.delete()

### 6.0  Optional exercises to the user 

 1. Holoviews works with a wide range of plotting libraries, Bokeh, matplotlib, plotly, mpld3, pygal to name a few. As an exercise, try changing the Holoviews plotting library to be `matplotlib` instead of `bokeh` in the `Setup` cell at the beginning of the notebook with `hv.extension('matplotlib')`. You will see the holoviews + matplotlib icons displayed when the library is loaded successfully. Run the cells in section 2.1 again and compare the outputs. Try again with some other plotting library. Don't forget to set the plotting library back to whichever you prefer to use for the rest of this tutorial.
  
 2. Try writing a different callback function to use in section 5.2. 